In [11]:
import numpy as np

from human_terms_network import Human_Terms_Network
from dataset_load import *
from lr_baseline import LRBaseline

In [2]:
path = r"..\..\data\reviews_Amazon_Instant_Video_5.json.gz"

X, y = extract_review_amazon(path, 'reviewText')

y_label = y
# discard the review with 3. [1,2] = 0, [4,5] = 0
neutral_indices = np.where(y_label == 3)[0]
y_label[y_label<3] = 0
y_label[y_label>3] = 1

X_discarded = np.delete(X,neutral_indices)
y_discarded = np.delete(y_label, neutral_indices)

from sklearn.model_selection import train_test_split

X_train_split, X_test_split, y_train_amazon, y_test_amazon = train_test_split(
    X_discarded, y_discarded)

X_train_corpus_update = update_corpus_contraction(X_train_split)
X_test_corpus_update = update_corpus_contraction(X_test_split)



(75, 2)
corpus update start
corpus update end

(75, 2)
corpus update start
corpus update end



In [3]:
word_list, connotation = load_unigrams('./amazon-video-unigrams.txt', X_train_corpus_update, y_train_amazon)

In [4]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_corpus_update, X_test_corpus_update, 
                                                          word_list, connotation)

In [5]:
av_baseline = LRBaseline(X_train_corpus_update, X_test_corpus_update, y_train_amazon, y_test_amazon, 
                           human_terms=word_list)

In [6]:
# (train, test)

print(av_baseline.baseline())
print(av_baseline.baseline(penalty='l1'))
print(av_baseline.human_terms_baseline())
print(av_baseline.human_terms_baseline(penalty='l2'))

(0.9545012953367875, 0.9287188828172435)
(0.9542179404145078, 0.929204614450516)
(0.9021615932642487, 0.9061323618700667)
(0.9021615932642487, 0.9061323618700667)


In [7]:
X_train = av_baseline.baseline_cv.transform(X_train_corpus_update)
X_test = av_baseline.baseline_cv.transform(X_test_corpus_update)
y_train = y_train_amazon
y_test = y_test_amazon

In [12]:
amazon_htm = Human_Terms_Network(input_shape=X_train.shape[1],
                               human_terms_shape=len(word_list),
                                domain='amazon')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1645)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 1646      
Total params: 1,646
Trainable params: 1,646
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1645)         0                                            
_________________________________________

In [13]:
amazon_htm.set_data(X_train, X_test, y_train_agreement, y_test_agreement, y_train, y_test)

In [10]:
amazon_htm.train(epochs=10)

Train on 16551 samples, validate on 8153 samples
Epoch 1/10
16551/16551 [==============================] - 44s 3ms/step - loss: 0.3508 - mean_absolute_error: 0.3600 - acc: 0.8124 - val_loss: 0.2815 - val_mean_absolute_error: 0.3365 - val_acc: 0.8122
Epoch 2/10
16551/16551 [==============================] - 43s 3ms/step - loss: 0.2528 - mean_absolute_error: 0.3161 - acc: 0.8250 - val_loss: 0.2561 - val_mean_absolute_error: 0.3072 - val_acc: 0.8332
Epoch 3/10
15867/16551 [===========================>..] - ETA: 1s - loss: 0.2282 - mean_absolute_error: 0.2923 - acc: 0.8401

KeyboardInterrupt: 

In [ ]:
amazon_htm.combined.evaluate([X_test, amazon_htm.y_test_agreement], amazon_htm.y_test)

In [ ]:
amazon_htm.test(reject=True)

In [ ]:
amazon_htm.X_test.shape

In [ ]:
amazon_htm.y_train_agreement.shape